<a href="https://colab.research.google.com/github/SY-256/anomaly_detection/blob/main/notebook/chapter4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1次元非正規分布の最尤推定による異常検知
- ホテリング理論による異常検知手法は、データが正規分布に従うことを前提としている
- 正規分布以外の1次元確率分布を用いた異常検知の手法

## A. データの分布形状から確率分布の種類を選択する
適用する確率分布の種類を選択する

データの分布の形状に注目する
- 分布の山の数
- 分布の左右対称性
- 分布の裾の重さ

＋AICやBICといった定量的な評価指標を組み合わせる

### 分布の山の数

正規分布や対数正規分布のような主要な確率分布の多くは、山が一つの形状（単峰性）を持つ。そのため、データを可視化した際にヒストグラムで山が2つ以上ある場合は単峰性の分布をそのまま適用できない。

### 分布の左右対称性

分布形状が左右対称化は、確率分布の選択に大きく影響を与える。具体的には、左右対称であれば正規分布やスチューデントのt分布が、左右非対称であれば対数正規分布やガンマ分布が有力な候補になる。定量的に評価するためには平均値と中央値の差や、歪度が利用できる

### 分布の裾の重さ

裾の重い分布（heavy-tailed distribution）とは、分布端部での確率密度の減衰が緩やかで、外れ値が多く出現する分布を指す。定量的に評価するためには、尖度が利用できる。正規分布よりもt分布の方が、ガンマ分布よりも対数正規分布の方が裾が重いため、裾の重さに応じてこれらの分布を絞り込むこともできる。正規分布の尖度は3なので、3より大きいかが基準になる。

In [ ]:
# ガンマ分布の最尤推定による異常検知の実装例（学習）
import numpy  as np
from scipy import stats

##### 学習データの読み込みと前処理 #####
# 学習用サンプルとして形状母数パラメータk=3、
# 尺度母数パラメータθ=2のガンマ分布からデータを1000個生成
x_train = stats.gamma.rvs(a=3, scale=2, size=1000, random_state=42)

##### 学習ステップ1. 正常のモデルを作成する #####
# ガンマ分布を最尤推定（オフセットlocは固定）
params = stats.gamma.fit(x_train, floc=0)
k = params[0] # 形状母数パラメータk
theta = params[2] # 尺度母数パラメータθ

##### 学習ステップ2. 異常を表す指標（異常度）を定義する #####
# 式のみ定義

##### 学習ステップ3. 異常度に閾値を設ける #####
TARGET_FP_RATE = 0.0027 # ターゲットとする誤報率（正規分布の3σ相当=0.0027）
# 学習データの確率密度を求める
x_train_pd = stats.gamma.pdf(x_train, a=k, scale=theta, loc=0)
x_train_anom_score = -np.log(x_train_pd) # 異常度を求める（確率密度関数の負の対数尤度）

# 異常度の分位点から閾値を算出
a_th = np.quantile(x_train_anom_score, 1-TARGET_FP_RATE)

##### 学習で求めたパラメータをすべて表示 #####
print(f"k={k}")
print(f"theta={theta}")
print(f"a_th={a_th}")

データのヒストグラムと最尤推定したガンマ分布を重ねて表示

In [ ]:
# データのヒストグラムと最尤推定したカイ二乗分布を重ねて表示
import matplotlib.pyplot as plt

plt.hist(x=x_train, color="#bbbbbb", bins="sturges", density=True)
xline = np.linspace(-2, 20, 200)
estimated_dist = stats.gamma.pdf(xline, a=k, scale=theta)
plt.plot(xline, estimated_dist, color="#111111")
plt.xlim(left=0)

推定したガンマ分布がデータにフィットしていることがわかる

## 推論

学習フェーズで求めたパラメータと閾値を用いて、推論データに対する異常度の算出と異常判定を行う

In [ ]:
# ガンマ分布の最尤推定による異常検知の実装例（推論）
import pandas as pd

##### 学習したパラメータを記載 #####
K=3.171819749395545 # 形状母数パラメータk
THETA=1.9405272720011781 # 尺度母数パラメータθ
A_TH=6.341744806739121 # 異常度のしきい値

##### 推論データの読み込みろ前処理 #####
# 推論用の正常データとして形状母数パラメータk=3、
# 尺度母数パラメータθ=2のガンマ分布からデータを500個生成
x_inference_norm = stats.gamma.rvs(a=3, scale=2, size=500, random_state=42)
df_inference_norm = pd.DataFrame(x_inference_norm, columns=["x"])
df_inference_norm["label"] = "normal"
# 推論用の異常データとして平均20、標準偏差2の正規分布からデータを20個生成
x_inference_anom = stats.norm.rvs(loc=20, scale=2, size=20, random_state=42)
df_inference_anom = pd.DataFrame(x_inference_anom, columns=["x"])
df_inference_anom["label"] = "anomaly"
# 推論用の正常データと異常データを合体
df_inference = pd.concat([
    df_inference_norm,
    df_inference_anom
], axis=0)
df_inference = df_inference.reset_index(drop=True)
x_inference = df_inference["x"].to_numpy()

##### 推論を実行 #####
# 異常度を算出
# 学習済みモデルから確率密度を求める
x_inference_pd = stats.gamma.pdf(x_inference, a=K, scale=THETA, loc=0)
anomaly_scores = -np.log(x_inference_pd) # 異常度を求める
# 閾値により異常の有無を判定
pred = np.where(anomaly_scores > A_TH, "anomaly", "normal")
# 推論結果を表示
print(pred)

推論データの異常度のヒストグラムと閾値（Threshold）を重ねてプロット

In [ ]:
# 推論データの異常度のヒストグラムと閾値を重ねてプロット（正常データと異常データを色分け）
import matplotlib.pyplot as plt
import seaborn as sns

# 異常度をDataFrameに列として追加
df_inference["anomaly_score"] = anomaly_scores
# 描画用のFigureとAxesを生成
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

# 正常データのヒストグラム描画
df_inference_norm = df_inference[df_inference["label"] == "normal"]
sns.histplot(data=df_inference_norm["anomaly_score"], binwidth=0.4,
             color="#bbbbbb", ax=ax, stat="density", label="normal")
# 異常データのヒストグラム描画
df_inference_anom = df_inference[df_inference["label"] == "anomaly"]
sns.histplot(data=df_inference_anom["anomaly_score"], binwidth=0.4,
             color="#111111", ax=ax, stat="density", label="anomaly")
# 閾値を描画
ax.vlines(A_TH, 0, 0.8, color="black", lw=2)
ax.text(A_TH, 0.8, f"Threshold={round(A_TH, 4)}",
        verticalalignment="bottom", horizontalalignment="center")
# 凡例を表示
ax.legend()
# グラフ表示
plt.show()

ある程度精度良く推定できている